Credit: https://asperbrothers.com/blog/question-answering-python/

In [ ]:
!gdown --id '1f8hw-zkYBtX7RkF4m8aAZ9Tn-s7gkluo'
!gdown --id '1-G9K6yc_qqNVsuz5jZxdX-SYoGeRUPi7'

In [ ]:
import pandas as pd
import os

wiki_documents = pd.read_csv('wiki_documents.csv')
qas_t2 = pd.read_csv('qas_t2.csv')
wiki_documents.head()

,file_name,article_id,path,content
0,11781.txt,11781,/content/wiki-documents-nsc/11781.txt,วินสตัน เชอร์ชิล เซอร์วินสตัน ลีโอนาร์ด สเปนเซ...
1,65293.txt,65293,/content/wiki-documents-nsc/65293.txt,ฝ้าย ฝ้าย เป็นพันธุ์ไม้สกุลกอสไซเพียมในวงศ์ชบา...
2,685790.txt,685790,/content/wiki-documents-nsc/685790.txt,"คำวิเศษ ""คำวิเศษ"" เป็นซิงเกิลของวงดนตรีเครสเชน..."
3,530931.txt,530931,/content/wiki-documents-nsc/530931.txt,ริโอบัมบา ริโอบัมบา () หรือชื่อเต็ม ซานเปโดรเด...
4,236219.txt,236219,/content/wiki-documents-nsc/236219.txt,แตงกวา แตงกวา หรือ แตงร้าน เป็นไม้เลื้อยในวงศ์...


In [ ]:
qas_t2.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id
0,15001,2,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,ไม่ใช่,NaN,NaN,265515
1,15002,2,แองคิโลซอรัสเป็นไดโนเสาร์ที่เล็กที่สุดในสกุล แ...,ไม่ใช่,NaN,NaN,319289
2,15003,2,ลูกตุ้มขนาดใหญ่ของ แองคิโลซอรัส เป็นส่วนกระดูก...,ไม่ใช่,NaN,NaN,319289
3,15004,2,แอบถ่าย หรือโรคถ้ำมองเป็นความสนใจหรือข้อประพฤต...,ไม่ใช่,NaN,NaN,773798
4,15005,2,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์เป็นการแข่ง...,ไม่ใช่,NaN,NaN,358523


In [ ]:
qas_t2.answer.value_counts()

ไม่ใช่    1006
ใช่        994
Name: answer, dtype: int64

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(list(qas_t2['answer']))
qas_t2['labels'] = le.transform(qas_t2['answer'])
qas_t2.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,labels
0,15001,2,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,ไม่ใช่,NaN,NaN,265515,1
1,15002,2,แองคิโลซอรัสเป็นไดโนเสาร์ที่เล็กที่สุดในสกุล แ...,ไม่ใช่,NaN,NaN,319289,1
2,15003,2,ลูกตุ้มขนาดใหญ่ของ แองคิโลซอรัส เป็นส่วนกระดูก...,ไม่ใช่,NaN,NaN,319289,1
3,15004,2,แอบถ่าย หรือโรคถ้ำมองเป็นความสนใจหรือข้อประพฤต...,ไม่ใช่,NaN,NaN,773798,1
4,15005,2,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์เป็นการแข่ง...,ไม่ใช่,NaN,NaN,358523,1


In [ ]:
qas_t2_merged = qas_t2.join(wiki_documents.set_index('article_id'), on='article_id')
qas_t2_merged.head(1)

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,labels,file_name,path,content
0,15001,2,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,ไม่ใช่,NaN,NaN,265515,1,265515.txt,/content/wiki-documents-nsc/265515.txt,ภาษาฮีบรูอาซเกนาซี ภาษาฮีบรูอาซเกนาซี (Ashkena...


In [ ]:
df = qas_t2_merged[['question', 'labels', 'content']]
df.columns

Index(['question', 'labels', 'content'], dtype='object')

In [ ]:
!pip install -qq datasets

     |████████████████████████████████| 311 kB 4.1 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 1.1 MB 55.8 MB/s 
     |████████████████████████████████| 133 kB 68.6 MB/s 
     |████████████████████████████████| 243 kB 87.1 MB/s 
     |████████████████████████████████| 271 kB 58.4 MB/s 
     |████████████████████████████████| 144 kB 78.2 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 


In [ ]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)
group1 = dataset.train_test_split(test_size=0.1)
group2 = group1['train'].train_test_split(test_size=0.2)
dataset = DatasetDict({
    'train': group2['train'],
    'test': group1['test'],
    'valid': group2['test']})
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'labels', 'content'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['question', 'labels', 'content'],
        num_rows: 200
    })
    valid: Dataset({
        features: ['question', 'labels', 'content'],
        num_rows: 360
    })
})

In [ ]:
!pip install transformers -qq

     |████████████████████████████████| 3.4 MB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 71.3 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 895 kB 73.5 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModel
from datasets import load_metric
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments

In [ ]:
pretrained = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained)

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    encoded = tokenizer(example['question'], example['content'], truncation=True, max_length=256)
    encoded['labels'] = example['labels']
    return encoded

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'labels', 'content', 'input_ids', 'attention_mask'],
        num_rows: 1440
    })
    test: Dataset({
        features: ['question', 'labels', 'content', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
    valid: Dataset({
        features: ['question', 'labels', 'content', 'input_ids', 'attention_mask'],
        num_rows: 360
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained)

Downloading:   0%|          | 0.00/921 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [ ]:
#https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [ ]:
!pip install -qq wandb

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 180 kB 89.7 MB/s 
     |████████████████████████████████| 143 kB 78.9 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 


In [ ]:
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
trainer = Trainer(model,
                  training_args,
                  train_dataset=tokenized_datasets['train'],
                  eval_dataset=tokenized_datasets['valid'],
                  data_collator=data_collator,
                  tokenizer=tokenizer)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: question, content.
***** Running training *****
  Num examples = 1440
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 450


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=450, training_loss=0.3309751722547743, metrics={'train_runtime': 202.3987, 'train_samples_per_second': 35.573, 'train_steps_per_second': 2.223, 'total_flos': 947208303820800.0, 'train_loss': 0.3309751722547743, 'epoch': 5.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])
y_pred = predictions.predictions.argmax(-1)
labels = predictions.label_ids

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: question, content.
***** Running Prediction *****
  Num examples = 200
  Batch size = 16


In [ ]:
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1])

In [ ]:
metric = load_metric('accuracy')

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
metric.compute(predictions=y_pred, references=predictions.label_ids)

{'accuracy': 0.855}

In [ ]:
trainer.save_model('output')
tokenizer.save_pretrained('output')

Saving model checkpoint to output
Configuration saved in output/config.json
Model weights saved in output/pytorch_model.bin
tokenizer config file saved in output/tokenizer_config.json
Special tokens file saved in output/special_tokens_map.json
tokenizer config file saved in output/tokenizer_config.json
Special tokens file saved in output/special_tokens_map.json


('output/tokenizer_config.json',
 'output/special_tokens_map.json',
 'output/tokenizer.json')

In [ ]:
model_ = AutoModelForSequenceClassification.from_pretrained('output', local_files_only=True)
tokenizer_ = AutoTokenizer.from_pretrained('output', local_files_only=True)

loading configuration file output/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "output",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "ENTAILMENT",
    "1": "NEUTRAL",
    "2": "CONTRADICTION"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "CONTRADICTION": 2,
    "ENTAILMENT": 0,
    "NEUTRAL": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 2

In [ ]:
trainer_ = Trainer(model_,
                   data_collator=data_collator,
                   tokenizer=tokenizer_)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
preds = trainer_.predict(tokenized_datasets['test'])
preds.predictions.argmax(-1)


The following columns in the test set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: question, content.
***** Running Prediction *****
  Num examples = 200
  Batch size = 8


array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1])